# RAG

# Initialisation

In [2]:
import os

# Vérifie si le code est exécuté sur Google Colab
if 'COLAB_GPU' in os.environ:
	# Commandes à exécuter uniquement sur Google Colab
	!git clone https://github.com/Zbrlaa/Rag
	%cd Rag
	!pip install -r requirements.txt
else:
	print("Pas sur Google Colab, ces commandes ne seront pas exécutées.")

fatal: destination path 'Rag' already exists and is not an empty directory.
/content/Rag


# Exercice 1 : Indexation

In [3]:
from langchain_community.document_loaders import DirectoryLoader

loader = DirectoryLoader("data/", use_multithreading=True)
documents = loader.load()
print(f"{len(documents)} documents")

65 documents


In [4]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
	chunk_size=1000,
	chunk_overlap=200,
	add_start_index=True,
	separators=["\n\n", "\n", " ", ""] #Ordre de priorité des caractères de découpe
)

all_splits = text_splitter.split_documents(documents)

print(f"Nombre de paragraphes : {len(all_splits)}")

Nombre de paragraphes : 8919


In [5]:
from langchain_community.embeddings import HuggingFaceEmbeddings

embedding_model = HuggingFaceEmbeddings(
	model_name="intfloat/multilingual-e5-base"
)

/tmp/ipython-input-464981128.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the `langchain-huggingface package and should be used instead. To use it run `pip install -U `langchain-huggingface` and import as `from `langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/387 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/57.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/694 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/418 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/200 [00:00<?, ?B/s]

In [6]:
from langchain_chroma import Chroma

vectorstore = Chroma.from_documents(
	documents=all_splits,
	embedding=embedding_model,
	persist_directory="./chroma_db"
)

print(f"Nombre d'éléments dans la base : {vectorstore._collection.count()}")

Nombre d'éléments dans la base : 8919


# Exercice 2 : Interrogation

In [7]:
def ask_base(query, k=3):
    # Recherche docs les plus proches et scores de distance
    docs_and_scores = vectorstore.similarity_search_with_score(query, k)

    return docs_and_scores

In [8]:
ma_query = "IA driving"

result = ask_base(ma_query, 2)

for doc, score in result:
	print(f"- Score de distance : {score:.4f}")
	print(f"- Source : {doc.metadata.get('source', 'Inconnue')}")
	print(f"- Start Index : {doc.metadata.get('start_index', 0)}")
	print("-" * 50)

- Score de distance : 0.3397
- Source : data/autres_articles/2412.18607v1.pdf
- Start Index : 8737
--------------------------------------------------
- Score de distance : 0.3576
- Source : data/autres_articles/2412.18604v1.pdf
- Start Index : 43319
--------------------------------------------------


# Exercice 3 : Prompt Template

In [9]:
from langchain_core.prompts import PromptTemplate

template = """Vous êtes un assistant spécialisé dans l'analyse de documents.
Utilisez les éléments de contexte suivants pour répondre à la question posée.
Si vous ne connaissez pas la réponse, dites simplement que vous ne savez pas, n'essayez pas d'inventer une réponse.

CONTEXTE :
{context}

QUESTION :
{question}

RÉPONSE :"""

rag_prompt = PromptTemplate(
	template=template,
	input_variables=["context", "question"]
)

print(rag_prompt.format(context="1", question="2"))

Vous êtes un assistant spécialisé dans l'analyse de documents.
Utilisez les éléments de contexte suivants pour répondre à la question posée.
Si vous ne connaissez pas la réponse, dites simplement que vous ne savez pas, n'essayez pas d'inventer une réponse.

CONTEXTE :
1

QUESTION :
2

RÉPONSE :


# Exercice 4 : chaîne RAG

In [10]:
import os
from google.colab import userdata
from langchain_google_genai import ChatGoogleGenerativeAI

os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash")

In [11]:
def ask_llm_with_rag(question, k=10):
	docs_and_scores = ask_base(question, k=k)

	context = "\n\n".join([doc.page_content for doc, score in docs_and_scores])

	prompt_final = rag_prompt.format(context=context, question=question)

	response = llm.invoke(prompt_final)

	return response.content

ask_llm_with_rag("IA driving ?")

'Oui, le contexte fournit de nombreuses informations sur la conduite autonome, ou "IA driving".\n\nIl aborde divers aspects de la conduite autonome :\n\n*   **Approches et concepts :**\n    *   La conduite de bout en bout ("End-to-end driving"), y compris l\'apprentissage par imitation conditionnel ([14], [2]), l\'apprentissage par imitation basé sur des modèles ([26]), et la conduite autonome orientée planification ([29]).\n    *   La simulation de véhicules autonomes, avec des plateformes comme Carla ([17]) et Navsim ([15]), qui est une simulation non réactive basée sur les données pour l\'évaluation comparative.\n    *   Des approches basées sur la vision pour la conduite autonome ([28]).\n    *   Des champs d\'attention neuronaux (NEAT, [12]) et la fusion de capteurs basée sur des transformeurs (Transfuser, [13]) pour la conduite autonome.\n    *   La conduite autonome vectorisée via la planification probabiliste (VADV2, [11]).\n\n*   **Modèles et systèmes spécifiques :**\n    *   

# Exercice 5 : mémoire

In [13]:
from langchain_core.messages import HumanMessage, AIMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

qa_prompt = ChatPromptTemplate.from_messages([
    ("system", "Vous êtes un assistant spécialisé dans l'analyse de documents. "
            "Utilisez le contexte fourni pour répondre. Si vous ne savez pas, dites-le.\n\n"
            "CONTEXTE :\n{context}"),
    MessagesPlaceholder(variable_name="chat_history"),
    ("human", "{input}"),
])

def ask_gemini_with_memory(question, history):
    docs_and_scores = ask_base(question, k=5)
    context = "\n\n".join([doc.page_content for doc, score in docs_and_scores])

    chain = qa_prompt | llm

    response = chain.invoke({
        "context": context,
        "chat_history": history,
        "input": question
    })

    return response.content

chat_history = []

# Première question
q1 = "IA driving ?"
res1 = ask_gemini_with_memory(q1, chat_history)
print(f"Utilisateur: {q1}\nGemini: {res1}\n")

# Maj manuelle
chat_history.append(HumanMessage(content=q1))
chat_history.append(AIMessage(content=res1))

# Deuxième question
q2 = "Quelles sont ses caractéristiques principales ?"
res2 = ask_gemini_with_memory(q2, chat_history)
print(f"Utilisateur: {q2}\nGemini (avec mémoire): {res2}")

Utilisateur: IA driving ?
Gemini: Oui, le document aborde de manière significative le sujet de la conduite assistée par IA (Intelligence Artificielle), plus spécifiquement la **conduite autonome**.

Voici les éléments clés qui le confirment :

*   **Section dédiée :** Une section entière est intitulée "2.3. End-to-end Autonomous Driving".
*   **Concepts centraux :** Le texte parle de "driving language model", "driving world modeling" et "end-to-end driving".
*   **Modèles et techniques :** Il mentionne l'apprentissage par imitation conditionnel ("conditional imitation learning" [14]), l'apprentissage par imitation basé sur des modèles ("Model-based imitation learning" [26]), des modèles du monde génératifs pour la conduite autonome ("generative world model for autonomous driving" [27]), et des approches de conduite autonome de bout en bout basées sur la vision ("end-to-end vision-based autonomous driving" [28]) ou orientées vers la planification ("Planning-oriented autonomous driving" 

# Exercice 6 : outil résumé

In [19]:
from langchain.tools import tool
from langchain_community.document_loaders import PyPDFLoader
import os

@tool
def recuperer_document_complet(nom_du_fichier: str) -> str:
    """
    Recherche un document spécifique dans le dossier data/ et retourne son contenu intégral
    """
    dossier_data = "data/"
    # Recherche récursive
    chemin_complet = None
    for root, dirs, files in os.walk(dossier_data):
        if nom_du_fichier in files:
            chemin_complet = os.path.join(root, nom_du_fichier)
            break

    if not chemin_complet:
        return f"Erreur : Le fichier {nom_du_fichier} est introuvable dans {dossier_data}."

    # Chargement selon l'extension
    if nom_du_fichier.endswith('.pdf'):
        loader = PyPDFLoader(chemin_complet)
        pages = loader.load()
        return "\n".join([p.page_content for p in pages])
    else:
        with open(chemin_complet, 'r', encoding='utf-8') as f:
            return f.read()

tools = [recuperer_document_complet]

In [18]:
from langchain_core.messages import HumanMessage

llm_with_tools = llm.bind_tools(tools)

nom_fichier_test = "2412.18607v1.pdf"

prompt = f"Peux-tu récupérer le contenu complet du fichier {nom_fichier_test} et m'en faire un résumé structuré en 5 points ?"

msg = llm_with_tools.invoke([HumanMessage(content=prompt)])

tool_call = msg.tool_calls[0]
content = recuperer_document_complet.invoke(tool_call["args"])

final_prompt = f"Voici le contenu intégral du document :\n\n{content}\n\nRéalise maintenant le résumé structuré demandé."
resultat = llm.invoke(final_prompt)

print(f"Résumé de {nom_fichier_test}\n")
print(resultat.content)

Résumé de 2412.18607v1.pdf

Voici un résumé structuré du document "DrivingGPT: Unifying Driving World Modeling and Planning with Multi-modal Autoregressive Transformers" :

---

**Titre :** DrivingGPT: Unifying Driving World Modeling and Planning with Multi-modal Autoregressive Transformers

**Auteurs et Affiliations :**
*   Yuntao Chen, Yuqi Wang, Zhaoxiang Zhang
*   New Laboratory of Pattern Recognition, Institute of Automation, Chinese Academy of Sciences
*   School of Artificial Intelligence, University of Chinese Academy of Sciences
*   Centre for Artificial Intelligence and Robotics, HKISI, CAS

**Problématique :**
Les modèles de monde pour la conduite autonome actuels, principalement basés sur la diffusion vidéo, sont efficaces pour la génération visuelle mais manquent de flexibilité pour intégrer d'autres modalités comme les actions de conduite. L'intégration transparente de la modélisation du monde et de la planification dans un cadre différentiable reste un défi technique maj